In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

startDate = '20201120'
endDate = '20201123'


readPath = r'\\192.168.10.34\random_backup\Kevin_zhenyu\orderLog\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
rawOrderLog = []
for thisDate, thisPath in zip(dateLs, dataPathLs):
    data = pd.read_csv(thisPath)
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

display('There are accounts with duplicated ticks:')
display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

# assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
#                                                    .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
#                                                                 'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
# if rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
#                                                    .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
#                                                                 'absOrderSize', 'internalId'], keep=False)].shape[0] != 0:
#     assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
#                                                    .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
#                                                                 'absOrderSize', 'internalId'], keep=False)]['accCode'].unique() == [8856])
#     print('8856 has wrong internalId')
#     assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
#             'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0)
#     if rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['vai'] == -1)].shape[0] != 0:
#         print(rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['vai'] == -1)][['date', 'colo', 'accCode', \
#             'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])   
# else:
#     print('we can use internalId in 8856')
#     assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
#             'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0)   

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0)  

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog['broker'] = orderLog['accCode'] // 100
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog["broker"])
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
# rawOrderLog = rawOrderLog[rawOrderLog['ars'] % 10 == 1]


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0) & \
         (orderLog['date'] == orderLog['date'].max())].groupby(['accCode'])['orderDirection'].size())
    orderLog = orderLog[orderLog['directNum'] == 1]

assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')
orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['mrsb90'] == '-'])
orderLog = orderLog[orderLog['mrsb90'] != '-']
orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['aaa'] == '-'])
orderLog = orderLog[orderLog['aaa'] != '-']
orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog = orderLog[~orderLog['ars'].isnull()]
orderLog = orderLog[orderLog['ars'] % 10 == 1]
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'There are accounts with duplicated ticks:'

Series([], Name: ars, dtype: int64)

'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
302656,20201120,zs_66_01,6634,2000558,-1,1,46910.0,0,0,-1.0,1.812806e+10
306222,20201120,zs_52_06,5269,2000610,-1,1,49400.0,0,0,-1.0,1.086587e+06
433148,20201120,zs_64_01,6480,2002548,-1,1,50434.0,0,0,-1.0,1.119890e+05
437258,20201120,zs_54_01,5470,2002571,-1,7,49023.0,0,0,-1.0,8.689071e+08
437401,20201120,zs_66_01,6634,2002571,-1,1,50925.0,0,0,-1.0,1.812808e+10
438417,20201120,zs_54_01,5474,2002574,-1,1,40443.0,0,0,-1.0,8.689045e+08
438990,20201120,zs_64_01,6480,2002588,-1,1,40612.0,0,0,-1.0,8.495400e+04
439184,20201120,zs_66_01,6634,2002591,-1,1,47104.0,0,0,-1.0,1.812806e+10
443022,20201120,zs_96_08,9685,2002611,-1,1,40493.0,0,0,-1.0,7.221449e+17
463510,20201120,zs_64_01,6480,2002732,-1,1,52199.0,0,0,-1.0,1.270590e+05


...

,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,...,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status


11. Orders with size > 80w or notional > 800w


C:\Users\win\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,...,orderDirection1,directNum,isMsg,status,orderNtl,m1,m2,exchange,tradeNtl,sta
797163,344808,20201123,1606113921669011,1606113921668957,zt_94_06,9561,1600660,0.0,144517000.0,-1,...,-1,1,0.0,2,142460.0,NaN,NaN,SSE,0.0,else
905263,396876,20201123,1606114609416617,1606114609416552,zs_96_08,974101,1603040,2331386.0,145649000.0,1,...,1,1,0.0,2,3692.0,NaN,NaN,SSE,0.0,else
1011023,723055,20201123,1606114626036092,1606114626035991,zt_52_04,527301,1603806,0.0,145645000.0,-1,...,-1,1,0.0,2,29896.0,NaN,NaN,SSE,0.0,else
1217815,905788,20201123,1606114613622185,1606114613622130,zs_66_01,6627,2002345,11772.0,145641370.0,-1,...,-1,1,1.0,2,4860.0,NaN,NaN,SZE,0.0,else
1218091,812823,20201123,1606114609108549,1606114609108462,zs_52_06,5287,2002345,11736324.0,145638250.0,-1,...,-1,1,1.0,2,143178.0,NaN,NaN,SZE,0.0,else
1301377,749655,20201123,1606114617420854,1606114617420792,zs_52_09,5225,2002742,0.0,145646300.0,-1,...,-1,1,1.0,2,11067.0,NaN,NaN,SZE,0.0,else


mrstaat  mrstauc
1000.0   0.0         54847
3000.0   1000.0     262820
         2000.0      19762
         3000.0       8682
11000.0  10000.0      1143
         20000.0       101
         30000.0        81
13000.0  11000.0     18541
         12000.0      1276
         13000.0       421
         21000.0       608
         22000.0       646
         23000.0       186
         31000.0       343
         32000.0       270
         33000.0       346
Name: date, dtype: int64

In [ ]:
orderLog

In [244]:
k = pd.read_pickle(r'L:\orderLog\result\marketPos\SZspeed_20201112.pkl')

In [245]:
k[k['accCode'] == 8856]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,agg,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,start_time,end_time,statusLs,TradePriceLs,TradeQtyLs,vai,ars,mrstaat,mrstauc,exchange,group,order,colo,accCode
913,12,9487143.0,1605148158656580,50930315,1.0,800.0,75000.0,42210716.0,1.0,102912010,0,0,0,2000012,1.209487e+09,20201112,1.605148e+15,1.605148e+15,"(0, 4)","(0, 75000)","(0, 800)",42209816.0,1.0,13000.0,11000.0,SZE,38955.0,112451.0,zs_88_04,8856.0
2230,25,3591101.0,1605145467110065,19857152,2.0,180.0,192700.0,497654.0,0.0,94420470,0,0,0,2000025,2.503591e+09,20201112,1.605145e+15,1.605145e+15,"(0, 3)","(0, 0)","(0, 0)",25845395.0,1.0,13000.0,11000.0,SZE,39299.0,112457.0,zs_88_04,8856.0
2280,25,12033832.0,1605150669133878,70429404,2.0,80.0,191800.0,1410931.0,0.0,111102490,0,0,0,2000025,2.512034e+09,20201112,1.605151e+15,1.605151e+15,"(0, 4)","(0, 191800)","(0, 80)",1410331.0,1.0,13000.0,11000.0,SZE,39289.0,112456.0,zs_88_04,8856.0
5018,62,17628354.0,1605161895789093,107959460,2.0,100.0,135000.0,3408810.0,1.0,141809120,0,0,0,2000062,6.217628e+09,20201112,1.605162e+15,1.605162e+15,"(0, 4)","(0, 135000)","(0, 100)",3387610.0,1.0,13000.0,11000.0,SZE,39955.0,112461.0,zs_88_04,8856.0
6116,90,8433585.0,1605148165303163,50986459,2.0,100.0,68300.0,2733594.0,1.0,102918660,0,0,0,2000090,9.008434e+09,20201112,1.605148e+15,1.605148e+15,"(0, 4)","(0, 68300)","(0, 100)",24406917.0,1.0,13000.0,11000.0,SZE,40228.0,112464.0,zs_88_04,8856.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204159,300376,3709806.0,1605145554271684,21331919,1.0,200.0,101100.0,8301000.0,1.0,94547630,0,0,0,2300376,3.003760e+13,20201112,1.605146e+15,1.605146e+15,"(0, 4)","(0, 101100)","(0, 200)",39462935.0,1.0,13000.0,11000.0,SZE,93389.0,112673.0,zs_88_04,8856.0
220497,300474,3209748.0,1605145366859330,18388388,1.0,100.0,617000.0,388605.0,1.0,94240220,0,0,0,2300474,3.004740e+13,20201112,1.605145e+15,1.605145e+15,"(0, 4)","(0, 617000)","(0, 100)",24406917.0,1.0,13000.0,11000.0,SZE,98322.0,112674.0,zs_88_04,8856.0
237811,300558,3028612.0,1605145187347874,15462107,1.0,100.0,1099900.0,857419.0,1.0,93940710,0,0,0,2300558,3.005580e+13,20201112,1.605145e+15,1.605145e+15,"(0, 4)","(0, 1099900)","(0, 100)",34211808.0,1.0,3000.0,1000.0,SZE,103372.0,112675.0,zs_88_04,8856.0
250283,300618,3514993.0,1605145493061666,20244914,1.0,100.0,688300.0,971171.0,1.0,94446420,0,0,0,2300618,3.006180e+13,20201112,1.605145e+15,1.605145e+15,"(0, 4)","(0, 688300)","(0, 100)",970871.0,1.0,13000.0,11000.0,SZE,106939.0,112677.0,zs_88_04,8856.0


In [219]:
rawOrderLog[['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']]

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
0,20201120,zs_88_04,8967,1600000,-1,2,-1.0,-2,2300,1619.0,1619.0
1,20201120,zs_88_04,8967,1600000,-1,4,-1.0,-2,2300,1619.0,1619.0
2,20201120,zs_88_04,8967,1600000,-1,4,-1.0,-2,2300,1619.0,1619.0
3,20201120,zs_88_04,8967,1600000,-1,2,-1.0,-2,600,2239.0,2239.0
4,20201120,zs_88_04,8967,1600000,-1,4,-1.0,-2,600,2239.0,2239.0
...,...,...,...,...,...,...,...,...,...,...,...
1586267,20201123,zs_94_05,9471,2300908,302,0,104939000.0,-1,1148,1672.0,-1.0
1586268,20201123,zs_96_08,9685,2300908,302,0,104915000.0,-1,100,552.0,-1.0
1586269,20201123,zs_96_08,9685,2300908,302,0,104927000.0,-1,200,553.0,-1.0
1586270,20201123,zs_96_08,9685,2300908,302,0,104939000.0,-1,448,554.0,-1.0


In [220]:
orderLog[['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']]

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
0,20201120,zs_88_04,8967,1600000,26173742.0,0,110512000.0,-2,2300,1619.0,-1.0
1,20201120,zs_88_04,8967,1600000,26173742.0,2,-1.0,-2,2300,1619.0,1619.0
2,20201120,zs_88_04,8967,1600000,26173742.0,4,-1.0,-2,2300,1619.0,1619.0
3,20201120,zs_88_04,8967,1600000,26173742.0,4,-1.0,-2,2300,1619.0,1619.0
4,20201120,zt_88_06,8970,1600000,26173742.0,0,110512000.0,-2,2300,2133.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
1584536,20201123,zs_52_06,5287,2300815,1554179.0,2,-1.0,1,200,5802.0,1556542.0
1584537,20201123,zs_52_06,5287,2300815,1554179.0,4,-1.0,1,200,5802.0,1556542.0
1584538,20201123,zs_52_06,5287,2300815,1554479.0,0,144942180.0,1,200,5805.0,-1.0
1584539,20201123,zs_52_06,5287,2300815,1554479.0,2,-1.0,1,200,5805.0,1556806.0


In [121]:
test.groupby(['colo', 'accCode', 'secid', 'orderDirection', 'absOrderSize'])['internalId'].nunique()[test.groupby(['colo', 'accCode', 'secid', 'orderDirection', 'absOrderSize'])['internalId'].nunique() != 1]

colo      accCode  secid      orderDirection  absOrderSize
zs_52_06  5222     2000020.0  -1.0            100.0           2
                                              600.0           2
                   2000032.0  -1.0            800.0           3
                   2000037.0  -1.0            200.0           3
                                              300.0           2
                                                             ..
zt_96_09  968501   1603730.0  -1.0            700.0           3
                   1603896.0  -1.0            100.0           2
                                              1700.0          2
                               1.0            500.0           3
                   1603920.0   1.0            100.0           4
Name: internalId, Length: 26755, dtype: int64

In [125]:
test[(test['accCode'] == 8856) & (test['orderDirection'] == -1) & (test['absOrderSize'] == 600) & (test['secid'] == 2000830)]

,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,absOrderSize,updateType,tradePrice,absFilledThisUpdate,absOrderSizeCumFilled,tradeId,orderSysId,aaa,internalId,ars,mse,mrsb300,mfe,mra100,session,hee,mt,sequenceNo,mrsb90,mrss90,ms,threadId,mta,mrstauc,orderPrice,finalState,mrrlma,l4tr,mrstaum,clock,cancellationPending,mrstaat,mrss300,underlyingIndex,ApplSeqNum,cfe,mrm,mrb100,orderId
196072,196072,20201119,1.605749e+15,1.605749e+15,zs_88_04,8856,2000830.0,788797.0,93000030.0,-1.0,600.0,0.0,-1.00,0.0,0.0,NaN,NaN,0.000906,26132868.0,1.0,100.0,-1.0,-1.0,1310.0,0,-0.000100,0.0,4370364.0,-0.002875,0.000906,09:30:13.395442,269440.0,-0.000734,0.0,13.08,0.0,0.0,0.0,0.0,1.605749e+15,0.0,1000.0,-1.0,905,370038.0,15220.0,-0.000734,1308.0,-1.0
196073,196073,20201119,1.605749e+15,1.605749e+15,zs_88_04,8856,2000830.0,-1.0,-1.0,-1.0,600.0,2.0,13.08,0.0,0.0,NaN,AE60000004,-1.000000,26132868.0,-1.0,0.0,-1.0,-1.0,1310.0,0,-1.000000,-1.0,4370364.0,-0.002875,0.000906,09:30:13.406442,269507.0,-1.000000,0.0,13.08,0.0,0.0,0.0,0.0,1.605749e+15,0.0,1000.0,-1.0,905,0.0,15220.0,-0.000734,1308.0,5.0
196101,196101,20201119,1.605749e+15,1.605749e+15,zs_88_04,8856,2000830.0,-1.0,34200.0,-1.0,600.0,1.0,-1.00,0.0,0.0,NaN,AE60000004,-1.000000,26132868.0,2.0,100.0,-1.0,-1.0,1304.0,0,-1.000000,-1.0,4600040.0,-0.001856,-0.001188,09:30:14.407442,269440.0,-1.000000,0.0,13.08,0.0,0.0,0.0,0.0,1.605749e+15,1.0,1000.0,-1.0,905,0.0,28402.0,-0.000882,1300.0,5.0
196102,196102,20201119,1.605749e+15,1.605749e+15,zs_88_04,8856,2000830.0,-1.0,-1.0,-1.0,600.0,3.0,13.08,0.0,0.0,NaN,NaN,-1.000000,26132868.0,-1.0,0.0,-1.0,-1.0,1304.0,0,-1.000000,-1.0,4600040.0,-0.001856,-0.001188,09:30:14.411442,269507.0,-1.000000,0.0,13.08,1.0,0.0,0.0,0.0,1.605749e+15,0.0,1000.0,-1.0,905,0.0,28402.0,-0.000882,1300.0,5.0
196130,196130,20201119,1.605749e+15,1.605749e+15,zs_88_04,8856,2000830.0,1281900.0,93001290.0,-1.0,600.0,0.0,-1.00,0.0,0.0,NaN,NaN,0.001666,26132868.0,1.0,100.0,-1.0,-1.0,1304.0,0,0.000131,0.0,4968682.0,-0.003499,0.001666,09:30:16.090442,269440.0,-0.000872,0.0,13.02,0.0,0.0,0.0,0.0,1.605749e+15,0.0,1000.0,-1.0,905,464899.0,43118.0,-0.000872,1302.0,-1.0
196131,196131,20201119,1.605749e+15,1.605749e+15,zs_88_04,8856,2000830.0,-1.0,-1.0,-1.0,600.0,2.0,13.02,0.0,0.0,NaN,AE6000000Q,-1.000000,26132868.0,-1.0,0.0,-1.0,-1.0,1304.0,0,-1.000000,-1.0,4968682.0,-0.003499,0.001666,09:30:16.094442,269507.0,-1.000000,0.0,13.02,0.0,0.0,0.0,0.0,1.605749e+15,0.0,1000.0,-1.0,905,0.0,43118.0,-0.000872,1302.0,27.0
196132,196132,20201119,1.605749e+15,1.605749e+15,zs_88_04,8856,2000830.0,-1.0,-1.0,-1.0,600.0,4.0,13.02,600.0,600.0,1.02e+14,NaN,-1.000000,26132868.0,-1.0,0.0,-1.0,-1.0,1304.0,0,-1.000000,-1.0,4968682.0,-0.003499,0.001666,09:30:16.094442,269507.0,-1.000000,0.0,13.02,1.0,0.0,0.0,0.0,1.605749e+15,0.0,1000.0,-1.0,905,0.0,50930.0,-0.000872,1302.0,27.0


In [126]:
test[(test['accCode'] == 8856) & (test['updateType'] != 0)]['vai'].unique() == [-1]

array([ True])

In [26]:
orderLog[(orderLog['updateType'] == 0) & (orderLog['internalId'] == 355) & (orderLog['secid'] == 2000957)]

,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,absOrderSize,updateType,tradePrice,absFilledThisUpdate,absOrderSizeCumFilled,tradeId,orderSysId,aaa,internalId,ars,mse,mrsb300,mfe,mra100,session,hee,mt,sequenceNo,mrsb90,mrss90,ms,threadId,mta,mrstauc,orderPrice,finalState,mrrlma,l4tr,mrstaum,clock,cancellationPending,mrstaat,mrss300,underlyingIndex,ApplSeqNum,cfe,mrm,mrb100,orderId,test,t1
998,998,20201106,1.604630e+15,1.604630e+15,zs_92_01,9208,2000957.0,26693901.0,102515790.0,1.0,500.0,0.0,-1.0,0.0,0.0,NaN,NaN,0.003011,355.0,1.0,100.0,-1.0,-1.0,786.0,0,0.002275,0.0,57359579.0,0.003011,-0.004340,10:25:19.281323,11298.0,0.001677,0.0,7.86,0.0,0.0,0.0,0.0,1.604630e+15,0.0,1000.0,-1.0,852,10200546.0,145758.9761,0.001677,785.0,-1.0,105456,2.0
124700,124700,20201106,1.604639e+15,1.604639e+15,zt_88_02,8854,2000957.0,26693901.0,130459010.0,1.0,500.0,0.0,-1.0,0.0,0.0,NaN,NaN,0.000202,355.0,1.0,100.0,-1.0,-1.0,800.0,0,-0.000000,0.0,155777853.0,0.000202,-0.001588,13:04:53.743720,25775.0,0.000355,0.0,8.00,0.0,0.0,11301.0,0.0,1.604639e+15,0.0,1000.0,-1.0,852,15519914.0,259032.8900,0.000355,799.0,-1.0,105456,2.0


In [23]:
ret[(ret['secid'] == 2000957)]

,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,absOrderSize,updateType,tradePrice,absFilledThisUpdate,absOrderSizeCumFilled,tradeId,orderSysId,aaa,internalId,ars,mse,mrsb300,mfe,mra100,session,mrmum,hee,mt,sequenceNo,mrsb90,mrss90,ms,threadId,mta,mrstauc,orderPrice,finalState,mrrlma,l4tr,mrstaum,mrm25,clock,cancellationPending,mrstaat,mrss300,underlyingIndex,ApplSeqNum,cfe,mrm,mrb100,orderId,test,t1,order,firstUpdateType,insertNum,innerSeq,firstClock,indexCat,beta_60,mdStartClock,mdClockAtArrival,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s


In [8]:
pd.set_option('max_columns', 200)
orderLog[orderLog['accCode'] == 8856]

,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,absOrderSize,updateType,tradePrice,absFilledThisUpdate,absOrderSizeCumFilled,tradeId,orderSysId,aaa,internalId,ars,mse,mrsb300,mfe,mra100,session,hee,mt,sequenceNo,mrsb90,mrss90,ms,threadId,mta,mrstauc,orderPrice,finalState,mrrlma,l4tr,mrstaum,clock,cancellationPending,mrstaat,mrss300,underlyingIndex,ApplSeqNum,cfe,mrm,mrb100,orderId
85771,85771,20201106,1.604626e+15,1.604626e+15,zs_88_04,8856,2000998.0,61100.0,93000030.0,1.0,200.0,0.0,-1.00,0.0,0.0,NaN,NaN,0.001047,9.562677e+08,1.0,100.0,-1.000000,-1.0,1732.0,0,0.000298,0.0,7184420.0,0.001047,-0.001569,09:29:59.917281,313132.0,0.003533,0.0,17.32,0.0,0.0,0.0,0.0,1.604626e+15,0.0,1000.0,-1.000000,905,378290.0,212606.0,0.003533,1731.0,-1.0
85772,85772,20201106,1.604626e+15,1.604626e+15,zs_88_04,8856,2000998.0,-1.0,-1.0,1.0,200.0,2.0,17.32,0.0,0.0,NaN,AE60000000,-1.000000,9.562677e+08,-1.0,0.0,-1.000000,-1.0,1732.0,0,-1.000000,-1.0,7184420.0,0.001047,-0.001569,09:29:59.963281,313199.0,-1.000000,0.0,17.32,0.0,0.0,0.0,0.0,1.604626e+15,0.0,1000.0,-1.000000,905,0.0,212606.0,0.003533,1731.0,1.0
85773,85773,20201106,1.604626e+15,1.604626e+15,zs_88_04,8856,2000998.0,-1.0,-1.0,1.0,200.0,4.0,17.32,200.0,200.0,1.04e+14,NaN,-1.000000,9.562677e+08,-1.0,0.0,-1.000000,-1.0,1732.0,0,-1.000000,-1.0,7184420.0,0.001047,-0.001569,09:29:59.963281,313199.0,-1.000000,0.0,17.32,1.0,0.0,0.0,0.0,1.604626e+15,0.0,1000.0,-1.000000,905,0.0,212606.0,0.003533,1731.0,1.0
85774,85774,20201106,1.604626e+15,1.604626e+15,zs_88_04,8856,2002511.0,1834398.0,93006630.0,1.0,400.0,0.0,-1.00,0.0,0.0,NaN,NaN,0.002117,4.227849e+09,1.0,100.0,-1.000000,-1.0,2235.0,0,0.000547,0.0,9035003.0,0.002117,-0.002971,09:30:08.188282,313133.0,0.004376,0.0,22.35,0.0,0.0,0.0,0.0,1.604626e+15,0.0,1000.0,-1.000000,905,621121.0,203665.0,0.004376,2233.0,-1.0
85775,85775,20201106,1.604626e+15,1.604626e+15,zs_88_04,8856,2002511.0,-1.0,-1.0,1.0,400.0,2.0,22.35,0.0,0.0,NaN,AE60000001,-1.000000,4.227849e+09,-1.0,0.0,-1.000000,-1.0,2235.0,0,-1.000000,-1.0,9035003.0,0.002117,-0.002971,09:30:08.191282,313199.0,-1.000000,0.0,22.35,0.0,0.0,0.0,0.0,1.604626e+15,0.0,1000.0,-1.000000,905,0.0,203665.0,0.004376,2233.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86633,86633,20201106,1.604646e+15,1.604646e+15,zs_88_04,8856,2300134.0,-1.0,-1.0,1.0,100.0,4.0,12.62,100.0,100.0,1.02e+14,NaN,-1.000000,9.562677e+08,-1.0,0.0,-1.000000,-1.0,1262.0,0,-1.000000,-1.0,282132420.0,-1.000000,-1.000000,14:55:29.348212,313199.0,-1.000000,0.0,12.62,1.0,0.0,0.0,0.0,1.604646e+15,0.0,1000.0,-1.000000,905,0.0,44630.0,0.004526,1260.0,311.0
86634,86634,20201106,1.604646e+15,1.604646e+15,zs_88_04,8856,2002372.0,12256230.0,145538360.0,1.0,100.0,0.0,-1.00,0.0,0.0,NaN,NaN,0.001490,9.730531e+08,1.0,100.0,-0.000009,-1.0,1949.0,0,0.000378,0.0,282359020.0,0.001490,-0.002064,14:55:38.136213,313130.0,0.002014,11000.0,19.49,0.0,0.0,0.0,0.0,1.604646e+15,0.0,13000.0,-0.000504,905,23052691.0,42681.0,0.002014,1948.0,-1.0
86635,86635,20201106,1.604646e+15,1.604646e+15,zs_88_04,8856,2002372.0,-1.0,-1.0,1.0,100.0,2.0,19.49,0.0,0.0,NaN,AE600000BN,-1.000000,9.730531e+08,-1.0,0.0,-0.000126,-1.0,1951.0,0,-1.000000,-1.0,282359109.0,0.000333,-0.001754,14:55:38.139213,313199.0,-1.000000,11000.0,19.49,0.0,0.0,0.0,0.0,1.604646e+15,0.0,13000.0,-0.001412,905,0.0,42681.0,0.001989,1948.0,312.0
86636,86636,20201106,1.604646e+15,1.604646e+15,zs_88_04,8856,2002372.0,-1.0,-1.0,1.0,100.0,4.0,19.49,100.0,100.0,1.01e+14,NaN,-1.000000,9.730531e+08,-1.0,0.0,-0.000126,-1.0,1951.0,0,-1.000000,-1.0,282359109.0,0.000333,-0.001754,14:55:38.139213,313199.0,-1.000000,11000.0,19.49,1.0,0.0,0.0,0.0,1.604646e+15,0.0,13000.0,-0.001412,905,0.0,42681.0,0.001989,1948.0,312.0


In [2]:
orderLog[(orderLog['accCode'].isin([532801, 5332])) & (orderLog['updateType'] == 0)].groupby(['date', 'colo', 'exchange', 'accCode', 'mrstaat', 'mrstauc'])['sta'].size()

date      colo      exchange  accCode  mrstaat  mrstauc
20201120  zs_52_06  SSE       5332     1000.0   0.0         115
                                       3000.0   1000.0      467
                                                2000.0       39
                                                3000.0        6
                    SZE       5332     1000.0   0.0         171
                                       3000.0   1000.0      614
                                                2000.0        5
          zt_52_04  SSE       532801   1000.0   0.0          29
                                       3000.0   1000.0      139
                                                2000.0       17
                                                3000.0        2
20201123  zs_52_06  SSE       5332     1000.0   0.0         214
                                       3000.0   1000.0      940
                                                2000.0       84
                                                

In [12]:
rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] == 532801)]['orderDirection'].unique()

array([ 1, -1], dtype=int64)

In [11]:
rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] == 532801) & (rawOrderLog['orderDirection'].isin([-2, 2]))].shape[0]

0

In [15]:
import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz

def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

def build_query(start_date=None, end_date=None, index_id=None):
    query = {}
    def parse_date(x):
        if type(x) == int:
            return x
        elif type(x) == str:
            if len(x) != 8:
                raise Exception("date must be YYYYMMDD format")
            return int(x)
        elif type(x) == datetime.datetime or type(x) == datetime.date:
            return x.strftime("%Y%m%d").astype(int)
        else:
            raise Exception("invalid date type: " + str(type(x)))
    if start_date is not None or end_date is not None:
        query['date'] = {}
        if start_date is not None:
            query['date']['$gte'] = parse_date(start_date)
        if end_date is not None:
            query['date']['$lte'] = parse_date(end_date)
    def parse_symbol(x):
        if type(x) == int:
            return x
        else:
            return int(x)
    if index_id:
        if type(index_id) == list or type(index_id) == tuple:
            query['index_id'] = {'$in': [parse_symbol(x) for x in index_id]}
        else:
            query['index_id'] = parse_symbol(index_id)
    return query

def write_memb_data(db, name, df):
    collection = db[name]
    df1 = []
    for symbol in df['index_id'].unique():
        if symbol in collection.distinct('index_id'):
            symbol = int(symbol)
            m_ax = pd.DataFrame.from_records(collection.find({'index_id':{'$in':[symbol]}}).sort([('date',-1)]).skip(0).limit(1))['date'].values[0]
            df2 = df[(df['index_id'] == symbol) & (df['date'] > m_ax)]
            print(df2)
            df1 += [df2]
        else:
            print(symbol)
            df2 = df[(df['index_id'] == symbol)]
            print(df2)
            df1 += [df2]
    df1 = pd.concat(df1).reset_index(drop=True)
    df1 = df1.to_dict('records')
    collection.insert_many(df1) 

def delete_memb_data(db, name, start_date=None, end_date=None, index_id=None):
    collection = db[name]
    query = build_query(start_date, end_date, index_id)
    if not query:
        print('cannot delete the whole table')
        return None
    collection.delete_many(query)  

def read_stock_daily(db, name, start_date=None, end_date=None, skey=None, index_name=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_name is not None:
        query['index_name'] = {'$in': index_name}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'skey'])
    return df    

database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)


In [29]:
orderLog[(orderLog['accCode'] == 5374) & (orderLog['updateType'] == 0)].groupby('orderDirection')['date'].size()

orderDirection
-1    2708
 1    1624
Name: date, dtype: int64

In [16]:
IC = read_memb_daily(db1, 'index_memb', 20201120, 20201120, index_id=[1000905])['skey'].unique()
IF = read_memb_daily(db1, 'index_memb', 20201120, 20201120, index_id=[1000300])['skey'].unique()

In [34]:
set(orderLog[orderLog['accCode'] == 537401]['secid'].unique()) - set(IF)

set()

In [4]:
orderLog = orderLog[~orderLog['mrstauc'].isnull()]
orderLog['sta_uc'] = orderLog['sta'] + '_' + orderLog['mrstauc'].astype(int).astype(str)
orderLog["server"] = orderLog["colo"].apply(lambda x: x.split("_")[0] + x.split("_")[1] + x.split("_")[2])
orderLog["server_account"] = orderLog["server"] + '_' + orderLog['accCode'].astype('str')
checkLog = orderLog[orderLog['updateType'].isin([0, 3, 4])].reset_index(drop=True)
checkLog = checkLog[checkLog['duration'] <= 1e6]

checkLog["time"] = checkLog["clock"].apply(lambda x: x.strftime("%H%M%S")).astype(int)
# ol = checkLog[(checkLog["updateType"] == 0) & (~checkLog["ars"].isin([121, 221, 321, 131, 231, 331])) & (checkLog["time"]>= 93000)
#              & (checkLog["time"] < 93300)]["order"].unique()

sta_list = orderLog['sta_uc'].unique()

result = {}
for col in ['date', 'broker1', 'broker2', 'isFast']:
    result[col] = []
            
for i in sta_list:
    checkLog1 = checkLog[checkLog['sta_uc'] == i]
    checkLog1['accountNum'] = checkLog1.groupby(['date', 'secid', 'vai'])['server_account'].transform('nunique')
    checkLog1 = checkLog1[checkLog1['accountNum'] >= 2]
    checkLog1['cumFillSize'] = checkLog1.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
    checkLog1['status'] = np.where(checkLog1['cumFillSize'] == 0, 2, 
                                    np.where(checkLog1['cumFillSize'] < checkLog1['absOrderSize'], 1, 0))
    checkLog1 = checkLog1.groupby(['group', 'order'])[['date', 'server_account', 'secid', 'vai', 'status']].first().reset_index()

    for exchg in ['SZE']:

        print(exchg)

        if exchg == 'SSE':
            exchangeLog = checkLog1[checkLog1['secid'] < 2000000]
        else:
            exchangeLog = checkLog1[checkLog1['secid'] >= 2000000]

        for group, groupData in exchangeLog.groupby(['group']):
            date = groupData['date'].values[0]
            brokerLs = groupData['server_account'].values
            if len(brokerLs) == 0:
                continue
            statusLs = groupData['status'].values
            ixLs = [i for i in range(len(brokerLs))]
            for k, broker1, status1 in zip(ixLs[:-1], brokerLs[:-1], statusLs[:-1]):
                for broker2, status2 in zip(brokerLs[k+1:], statusLs[k+1:]):
                    if broker1 != broker2:
                        result['date'] += [date, date]
                        result['broker1'] += [broker1, broker2]
                        result['broker2'] += [broker2, broker1]

                        if status1 < status2:
                            result['isFast'] += [1, 0]
                        elif status1 > status2:
                            result['isFast'] += [0, 1]
                        else:
                            result['isFast'] += [-1, -1]                

result = pd.DataFrame(result)
result = result[result['isFast'] != -1]
result['count'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('count')
result['fasterSum'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
result['faster'] = result['fasterSum']/result['count']


summary = pd.DataFrame()
summaryCols = []
countCols = []
fasterCols = []
for date in result['date'].unique():
    a = result[result['date'] == date].reset_index(drop=True)
    a['count'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('count')
    a['fasterSum'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
    a['faster'] = a['fasterSum']/a['count']
    a = a.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
    a.columns = ['broker1', 'broker2', '%s count'%date, '%s faster'%date]
    summaryCols += ['%s count'%date, '%s faster'%date]
    countCols += ['%s count'%date]
    fasterCols += ['%s faster'%date]

    if summary.empty:
        summary = a.copy()
    else:
        summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')

a = result.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
a.columns = ['broker1', 'broker2', 'total count', 'total faster']
summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')
summaryCols = ['total count', 'total faster'] + summaryCols


savePath = r'L:\orderLog\result\relativeSpeed'
a = summary.groupby(['broker1', 'broker2'])[['total count', 'total faster']].first().reset_index()
a.to_csv(os.path.join(savePath, 'relativeSpeedAccount_%s_%s_%s.csv'%(startDate, endDate, exchg)), index=False)




countCols += ['total count']
fasterCols += ['total faster']
summary['b1'] = summary['broker1'].str[7:11].astype(int) // 100
summary['b2'] = summary['broker2'].str[7:11].astype(int) // 100
#     summary = summary.groupby(['broker1', 'broker2'])[summaryCols].first()
for col in countCols:
    summary[col] = summary[col].fillna(0)
    summary[col] = summary[col].astype('int64')

for col in fasterCols:
    summary[col] = summary[col].fillna(-1)
    summary[col] = summary[col].apply(lambda x: '%.0f%%'%(x*100))
    summary[col] = np.where(summary[col] == '-100%', ' ', summary[col])

#     summary = summary.rename(columns={"broker1":"account1", "broker2":"account2"})
#     summary = summary.groupby(['account1', 'account2'])[summaryCols].first()
#     from IPython.display import display, HTML
#     display(HTML(summary.iloc[:, :2].to_html()))
#     summary["tt"]=summary[["broker1", "broker2"]].min(axis=1).astype(str) + ' - ' + summary[["broker1", "broker2"]].max(axis=1).astype(str)
#     summary = summary.sort_values(by=["tt", "broker1"])
#     s1 = summary.groupby("tt").first().reset_index(drop=True)
s1 = summary
display(s1[(s1["broker1"] == 'zs5206_5332')])

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE


,broker1,broker2,20201120 count,20201120 faster,20201123 count,20201123 faster,total count,total faster,b1,b2
49,zs5206_5332,zs5208_5229,83,1%,161,7%,244,5%,53,52
50,zs5206_5332,zs5208_5377,25,100%,136,99%,161,99%,53,53
51,zs5206_5332,zs5208_5386,48,100%,167,99%,215,100%,53,53
52,zs5206_5332,zs5209_5289,84,1%,217,9%,301,7%,53,52
53,zs5206_5332,zs5209_5291,63,6%,115,14%,178,11%,53,52
54,zs5206_5332,zs8804_8856,8,12%,12,25%,20,20%,53,88
55,zs5206_5332,zs8804_8924,4,100%,0,,4,100%,53,89
56,zs5206_5332,zs8804_896702,3,100%,0,,3,100%,53,89
57,zs5206_5332,zs9403_9448,5,40%,5,40%,10,40%,53,94
58,zs5206_5332,zs9608_6237,1,100%,8,100%,9,100%,53,62


In [18]:
display(s1[(s1["broker1"] == 'zs5206_5332')])

,broker1,broker2,20201120 count,20201120 faster,20201123 count,20201123 faster,total count,total faster,b1,b2
0,zs5206_5332,zs5208_5386,3,100%,9,67%,12,75%,53,53
1,zs5206_5332,zs9608_6237,1,100%,0,,1,100%,53,62
2,zs5206_5332,zs9608_6282,1,100%,1,100%,2,100%,53,62
3,zs5206_5332,zs9608_9754,9,33%,62,15%,71,17%,53,97
4,zs5206_5332,zt5204_522201,21,0%,58,3%,79,3%,53,52
5,zs5206_5332,zt5204_522601,72,0%,81,2%,153,1%,53,52
6,zs5206_5332,zt5204_527301,119,2%,229,0%,348,1%,53,52
7,zs5206_5332,zt5204_532801,6,0%,15,0%,21,0%,53,53
8,zs5206_5332,zt5207_523001,117,1%,252,2%,369,1%,53,52
9,zs5206_5332,zt5210_522401,49,0%,81,1%,130,1%,53,52


In [5]:
checkLog = orderLog[(orderLog["updateType"] == 0)]
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
SSE = checkLog[checkLog['secid'] < 2000000]
SSE["exchange"] = "SH"

c1 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
c2 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].median().reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
c3 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].count().reset_index()
c4 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["date"].unique().str.len().reset_index()
c5 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].std().reset_index()

re1 = pd.merge(c3, c1, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1 = pd.merge(re1, c4, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = pd.merge(re1, c5, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency': 'std'})


# re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
re = pd.concat([re1]).reset_index(drop=True)


for col in ['median', '95 percentile']:
    re[col] = re[col].astype(int)
for col in ['std']:
    re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
from IPython.display import display, HTML

display(HTML(re[re['colo'].str[:5] == 'zt_52'].groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["count", "median", "95 percentile", "std"].first().to_html()))
display(HTML(re[re['colo'].str[:5] == 'zs_52'].groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["count", "median", "95 percentile", "std"].first().to_html()))

SZE = checkLog[checkLog['secid'] >= 2000000]
SZE["exchange"] = "SZ"

c1 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
c2 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].median().reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
c3 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].count().reset_index()
c4 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["date"].unique().str.len().reset_index()
c5 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].std().reset_index()

re1 = pd.merge(c3, c1, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1 = pd.merge(re1, c4, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = pd.merge(re1, c5, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency': 'std'})


# re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
re = pd.concat([re1]).reset_index(drop=True)


for col in ['median', '95 percentile']:
    re[col] = re[col].astype(int)
for col in ['std']:
    re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
from IPython.display import display, HTML

display(HTML(re[re['colo'].str[:5] == 'zs_52'].groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["count", "median", "95 percentile", "std"].first().to_html()))


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [32]:
import os
import glob
import datetime
import numpy as np
import pandas as pd

startDate = '20201117'
endDate = '20201123'
readPath = r'L:\orderLog\result\marketPos'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'SZspeed_***.pkl')))
dateLs = np.array([os.path.basename(i).split('.')[0].split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
checkData = []
for path in dataPathLs:
    data = pd.read_pickle(path)
    checkData += [data]
checkLog = pd.concat(checkData).reset_index(drop=True)   
checkLog['date'].unique()

array([20201119, 20201117, 20201118, 20201120, 20201123], dtype=int64)

In [33]:
checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog = checkLog[~checkLog['accCode'].isnull()]

In [34]:
cc1 = checkLog[(checkLog['accCode'] == 5332) | (checkLog['accCode'] // 100 == 52)]
cc1 = cc1.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
cc1 = cc1.reset_index(drop=True)
cc1['ordering'] = cc1.index
cc1['time_diff'] = cc1['clockAtArrival'] - cc1['start_time']
cc1['colo1'] = cc1['colo'].str[:2] + cc1['colo'].str[3:5] + cc1['colo'].str[6:8]
cc1['colo_broker'] = cc1['colo1'] + '_' + cc1["accCode"].astype(int).astype(str)
cc1['accCode'] = cc1['accCode'].fillna(0).astype(int)
cc1['sta'] = np.where(cc1['mrstaat'] == 1000, 'staone', np.where(
cc1['mrstaat'] == 3000, 'statwo', np.where(
cc1['mrstaat'] == 11000, 'stathree13', 'stathree23')))

checkLog1 = cc1[cc1['sta'] == 'statwo']
ol = pd.merge(cc1[cc1['sta'] == 'staone'], cc1[cc1['sta'] == 'statwo'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
checkLog2 = cc1[(cc1['sta'] == 'staone') & (~cc1['ordering'].isin(ol))]
checkLog2 = cc1[(cc1['sta'] == 'staone')]

from IPython.display import display, HTML
re1 = checkLog1.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog1.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['mrstaat', 'colo_broker', 'colo', '10%']], on=['mrstaat', 'colo_broker', 'colo'])
c1 = checkLog1.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['mrstaat', 'colo_broker', 'colo', '90%']], on=['mrstaat', 'colo_broker', 'colo'])

display(HTML(re1[['mrstaat', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['mrstaat', 'colo_broker']).first().to_html()))

from IPython.display import display, HTML
re1 = checkLog2.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog2.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['mrstaat', 'colo_broker', 'colo', '10%']], on=['mrstaat', 'colo_broker', 'colo'])
c1 = checkLog2.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['mrstaat', 'colo_broker', 'colo', '90%']], on=['mrstaat', 'colo_broker', 'colo'])

display(HTML(re1[['mrstaat', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['mrstaat', 'colo_broker']).first().to_html()))